### **Problem 2.1:**

Use Ridge regression to solve the regression problem in Example 2.1 as well as the classification problem in Example 2.2, also implement both closed-form and iterative approachs, compare the results of Ridge regression with those of linear regression.

In [2]:
# NAME: Asfar Waheed      Reg No: 2019-EE-260
#importing all libraries
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

#importing data
raw_data = pd.read_csv('boston.csv', header=None)
#reshaping raw data
data_rows = np.reshape(raw_data.to_numpy(), (506,14))
#separating features
data = data_rows[:,:13]
#separting output
target = data_rows[:,13]
# normalize input features to zero-mean and unit-variance
data = (data-np.mean(data, axis=0))/np.std(data, axis=0) 
print(data.shape)
print(target.shape)

(506, 13)
(506,)


In [3]:
# add a constant column of '1' to accomodate the bias 
data_wb = np.hstack((data, np.ones((data.shape[0], 1), dtype=data.dtype)))
I = np.eye(data_wb.shape[1])
#closed-form solution
w = np.linalg.inv(data_wb.T @ data_wb + 1*I) @ data_wb.T @ target
# calculate the mean square error in the training set
predict = data_wb @ w 
error = np.sum((predict - target)*(predict - target))/data.shape[0]
e1 = error #for further comparison
print(f'Mean square error for the closed-form solution: {error:.5f}')

Mean square error for the closed-form Ridge regression solution: 21.89784


In [4]:
class RidgeRegression():
    def __init__(self, learning_rate, iterations, L2_penality):  
        self.learning_rate = learning_rate        
        self.iterations = iterations        
        self.L2_penality = L2_penality            
    def fit(self, X, Y ):
        self.m, self.n = X.shape        
        self.W = np.zeros( self.n)
        self.b = 0        
        self.X = X        
        self.Y = Y
        for i in range(self.iterations):            
            self.update_weights()            
        return self
    def update_weights(self):           
        Y_pred = self.predict(self.X)
        dW = (-(2*(self.X.T).dot(self.Y - Y_pred)) + (2*self.L2_penality*self.W))/self.m     
        db = -2*np.sum(self.Y - Y_pred)/self.m     
        self.W = self.W - self.learning_rate * dW    
        self.b = self.b - self.learning_rate * db        
        return self 
    def predict(self, X) :    
        return X.dot( self.W ) + self.b

In [5]:
op = RidgeRegression(iterations = 1000, learning_rate = 0.01, L2_penality = 1)

op.fit( data_wb, target)
predict = op.predict(data_wb)
print("Mean squared error: %.5f" % mean_squared_error(target, predict))

Mean squared error Iterative Ridge regression: 21.96028


In [6]:
# Create linear regression object
l_regr = linear_model.LinearRegression()
# Train the model using the training set
l_regr.fit(data_wb, target)
# Make predictions using the same training set
predict = l_regr.predict(data_wb)
# The mean squared error
print("Mean squared error for linear regression: %.5f" % mean_squared_error(target, predict))

Mean squared error for linear regression: 21.89483
Mean squared error for Ridge regression: 21.89586


In [ ]:
# Using Ridge regression Scikit learn
l_Ridge = linear_model.Ridge(alpha = 1)
# Train the model using the training set
l_Ridge.fit(data_wb, target)
# Make predictions using the same training set
predict = l_Ridge.predict(data_wb)
# The mean squared error
print("Mean squared error for Ridge regression: %.5f" % mean_squared_error(target, predict))

**Solving Example 2.2**

In [7]:
from mnist import MNIST

mnist_loader = MNIST('MNIST')
train_data, train_label = mnist_loader.load_training()
test_data, test_label = mnist_loader.load_testing()
train_data = np.array(train_data, dtype='float')/255 # norm to [0,1]
train_label = np.array(train_label, dtype='short')
test_data = np.array(test_data, dtype='float')/255 # norm to [0,1]
test_label = np.array(test_label, dtype='short')
#add small random noise to avoid matrix singularity
train_data += np.random.normal(0,0.0001,train_data.shape) 
print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

(60000, 784) (60000,) (10000, 784) (10000,)


In [8]:
# prepare digits '3' and '8' for Ridge regression
digit_train_index = np.logical_or(train_label == 3, train_label == 8)
X_train = train_data[digit_train_index]
y_train = train_label[digit_train_index]
digit_test_index = np.logical_or(test_label == 3, test_label == 8)
X_test = test_data[digit_test_index]
y_test = test_label[digit_test_index]
# add a constant column of '1' to accomodate the bias
X_train = np.hstack((X_train, np.ones((X_train.shape[0], 1), dtype=X_train.dtype)))
X_test = np.hstack((X_test, np.ones((X_test.shape[0], 1), dtype=X_test.dtype)))
# convert labels: '3' => -1, '8' => +1
CUTOFF = 5 # any number between '3' and '8'
y_train = np.sign(y_train-CUTOFF)
y_test = np.sign(y_test-CUTOFF)
print(X_train.shape)
print(y_train)
print(X_test.shape)
print(y_test)

(11982, 785)
[-1 -1 -1 ...  1 -1  1]
(1984, 785)
[-1 -1 -1 ... -1  1 -1]


**Solving by Closed-form**

In [9]:
# use the Ridge regression closed-form solution
I = np.eye(X_train.shape[1])
# refer to the closed-form solution
w = np.linalg.inv(X_train.T @ X_train + 1*I) @ X_train.T @ y_train
# calculate the mean square error and Ridge regression accuracy on the training set
predict = X_train @ w 
error = np.sum((predict - y_train)*(predict - y_train))/X_train.shape[0]
print(f'Mean square error on training data for the closed-form solution: {error:.5f}')
accuracy = np.count_nonzero(np.equal(np.sign(predict),y_train))/y_train.size*100.0
print(f'accuracy on training data for the closed-form solution: {accuracy:.5f}%')

# calculate the mean square error and Ridge regression accuracy on the test set
predict = X_test @ w 
error = np.sum((predict - y_test)*(predict - y_test))/X_test.shape[0]
print(f'Mean square error on test data for the closed-form solution: {error:.5f}')
accuracy = np.count_nonzero(np.equal(np.sign(predict),y_test))/y_test.size*100.0
print(f'accuracy on test data for the closed-form solution: {accuracy:.5f}%')

Mean square error on training data for the closed-form Ridge regression solution: 0.20014
Ridge regression accuracy on training data for the closed-form solution: 96.78685%
Mean square error on test data for the closed-form Ridge regression solution: 0.22013
Ridge regression accuracy on test data for the closed-form solution: 96.01815%


**Solving by Iterative Manner**

In [10]:
op = RidgeRegression(iterations = 1000, learning_rate = 0.01, L2_penality = 2)
op.fit(X_train, y_train)
predict1 = op.predict(X_train)
print("Mean square error on training data solution: %.5f" % mean_squared_error(y_train, predict1))

predict2 = op.predict(X_test)
print("Mean square error on test data solution: %.5f" % mean_squared_error(y_test, predict2))

Mean square error on training data for the Iterative Ridge regression solution: 0.21094
Mean square error on test data for the Iterative Ridge regression solution: 0.21064


In [11]:
# Create linear regression object
l_regr = linear_model.LinearRegression()
# Train the model using the training set
l_regr.fit(X_train, y_train)
# Make predictions using the same training set
predict = l_regr.predict(X_train)
print("Mean squared error on training data for Linear regression: %.5f" % mean_squared_error(y_train, predict))
# Make predictions using the test set
predict = l_regr.predict(X_test)
print("Mean squared error on test data for Linear regression: %.5f" % mean_squared_error(y_test, predict))

Mean squared error on training data for Linear regression: 0.19602
Mean squared error on test data for Linear regression: 2.13765
Mean squared error on training data for Ridge regression: 0.20014
Mean squared error on test data for Ridge regression: 0.22013


In [ ]:
# Create Ridge regression object
Ridge = linear_model.Ridge()
# Train the model using the training set
Ridge.fit(X_train, y_train)
# Make predictions using the same training set
predict = Ridge.predict(X_train)
print("Mean squared error on training data : %.5f" % mean_squared_error(y_train, predict))
# Make predictions using the test set
predict = Ridge.predict(X_test)
print("Mean squared error on test data : %.5f" % mean_squared_error(y_test, predict))

### **Problem 2.2:**

Use LASSO to solve the regression problem in Example 2.1 as well as the classification problem in Example 2.2, compare the results of LASSO with those of linear regression and Ridge regression.

In [12]:
lasso = linear_model.Lasso(alpha = 1)
lasso.fit(data_wb, target)
predict = lasso.predict(data_wb)
print("Mean squared error: %.5f" % mean_squared_error(target, predict))

Mean squared error: 28.46465
